# Engineering Notebook Final Project

Goal: develop a replicated, distributed database system which supports both strongly consistent data and eventually consistent data. The idea is that writes and reads can be directed to any of the replicated databases, with consistency guarantees only for those tables/columns marked as strongly consistent.

Challenges brainstorm:
- We need to use non-sequenced primary keys to make sure that foreign keys can be made to work
- At what level do we implement the consistency requirements? Do we just create separate servers, each with their own database, that receive instructions via sockets?
- Queries that involve columns/tables of mixed consistency requirements are difficult
- Do we send the replication before or after the commit on the first replica? -> Has to be before to avoid rolling back a transaction
- Can every replica initiate a strongly consistent transaction or do we have a leader to whom those transactions are forwarded?
    - Edge case: a strongly consistent column only gets changed by `onupdate` (or `default`), especially bad in case where this is a timestamp (now why would you ever do that, I'm not sure)
- How can we do Python defaults at a transaction level (e.g. set the CURRENT_TIMESTAMP to be the same for all objects in a transaction without going back and forth to the database)?
- How can we make sure that only the columns/tables affected by a transaction are "locked" for the duration of a strong transaction?

If strongly consistent, need to get the majority/everyonen on board; then 
Weakly consistent; store locally; then pass onto all other replicas on whatever time we want; 

if we read from a strongly consistent thing; we have to also ask the leader; who will inform if it is still in the process of informing all replicas that a strongly consistent write is happening. 

what happens if you send the same transaction twice or to multiple machines; there should be a log on that machine if it did not go down or a different machine that is still up that help check if the thing has already been done it does not get done twice. 

leader election; the config.json file specifies possible ports; having one leader; if it goes down sometime then the first machine that detects this send a message starting from the machine with the lowest port number; and it gets elected as the leader. 


Load Balancer: not sure for now; think later

### April 19
- From a point of abstraction, the API should not be aware that it is dealing with multiple database replicas. So we either need to implement a separate `Session` class which monitors all read and write statements, and then sets the database accordingly, but this will also have to implement some consensus logic, which definitely shouldn't be a part of the API's code at all
- Alternative: instead of using a full SQL server, for now, build a very simple relational database ourselves, so that we can intercept all queries easily
    - We could re-use part of our wire protocol to convert from and to binary data
    - Potential complexity: transactions, subqueries, etc.; importantly, we'd also need to write a simple query language and parse it
    - If we restrict ourselves to only using insert, update, delete, and maybe begin/commit, we can already do a lot (but for example no foreign keys, multiple indexes, complex filters, etc.), and we could implement this using a REST API with GET, PATCH, POST requests for simplicity
- Second alternative: build a proxy database server through which all of the requests pass, and which forwards most of them
    - Complexities: this proxy will need to do a lot of parsing

### April 22
- Okay idk why I thought we need to build a whole database system from scratch (it was kinda fun doing this though) but we can just create a proxy which uses its own simple language but then a regular SQL database underneath
- For now, let's use a REST API but we can probably convert that to sockets for a little extra performance
- Next steps: implement the proxy with an outside interface for sending queries, and an inside interface for taking care of the communication between replicas
- Also, do we actually need to do strong select queries?
    - If we only require a majority (e.g. Raft), then yes (could there be a case where a replica which doesn't respond/agree drops that request and then takes a read request?)
    - If we require unanimous agreement, then no, since in that case all replicas would have been updated by the time that the client sends a read request
    - Generally: always problems with temporary partition failure, even with unanimous agreement

### April 23
- We need to send clock values along with every strong query for synchronization
- We don't need to check if the followers *accept* the query (since if the leader can do it, they should all be able to do it), but rather just confirm that they have done it
- Defaults and onupdate functions need to be implemented by the client connecting to the database (e.g. the API)

### April 25
- This parsing is actually a much bigger pain than I thought it would be; if we use a proxy, the problem is that Postgres for example uses some type of SSL encryption at the beginning to make the connection and the password transmission secure which is really annoying to take care of; if we use middleware, then the problem is that SQL uses relations to make the reference to objects easier and more efficient but this means that we can't just switch from one server to another because that would mean that we send information about relations which the database doesn't know about because we haven't established what that relation means (with that specific database). Now, we could just also transmit relation IDs but then the problem is that those are probably only unique within a specific session between one client and one database, and so there could be clashes and ambiguity
- So basically I think that unfortunately, just doing the whole separate, simpler REST protocol will be much easier

### April 26
- It is still an open problem in our code when a replica crashes between having implemented a query and returning the acceptance (this is why Raft does proposals and commits with a majority)


### April 29
- Our proxy needs to implement defaults and onupdates somehow
- As another challenge, for weak transactions, there is currently not really a guarantee that they will be eventually consistent at all: if we have two update queries with different values happening at the same time, they will just end up in an inconsistent state, since both replicas commit them to their own version, then they receive the others' version, and update again. If we want to avoid this, we need to keep track of a commit history for each *field* in our database along with operations, timestamps, and old + new values.
- This is probably what we actually want lol